In [7]:
from pathlib import Path
import os

import pandas as pd
import boto3

In [8]:
df = pd.read_csv("TrainingFollowUp.csv")
# df["UploadS3"] = False
# df["UploadS3Location"] = None
# df.to_csv("TrainingFollowUp.csv", index=False)

In [121]:

df.head()

,num_epochs,lr_generator,lr_discriminator,schedule,nb_step_discrim,nb_epoch_fade_in_new_layer,EMA_value,device,use_loss,momment_loss,...,channel_nb,pos_enc_dimension,self_attention,TrainingJobName,encoder_network_factor,DownloadStatus,TrainingJobStatus,SaveLocation,UploadS3,UploadS3Location
0,2500,0.00005,0.00005,[1000],1,2,0.2,gpu,lsgan,1,...,16,0,True,PSA-GAN-traffic-len16-29082021-010119-969,NaN,False,Failed,NaN,False,None
1,3500,0.00005,0.00005,"[1000, 2000]",1,2,0.2,gpu,lsgan,1,...,16,0,True,PSA-GAN-traffic-len32-29082021-010134-981,NaN,False,Failed,NaN,False,None
2,4500,0.00005,0.00005,"[1000, 2000, 3000]",1,2,0.2,gpu,lsgan,1,...,16,0,True,PSA-GAN-traffic-len64-29082021-010151-403,NaN,False,Failed,NaN,False,None
3,5500,0.00005,0.00005,"[1000, 2000, 3000, 4000]",1,2,0.2,gpu,lsgan,1,...,16,0,True,PSA-GAN-traffic-len128-29082021-010206-716,NaN,False,Failed,NaN,False,None
4,6500,0.00005,0.00005,"[1000, 2000, 3000, 4000, 5000]",1,2,0.2,gpu,lsgan,1,...,16,0,True,PSA-GAN-traffic-len256-29082021-010222-415,NaN,False,Failed,NaN,False,None


In [10]:
from pathlib import Path
import pandas as pd
import boto3
from collections import namedtuple
import time
import numpy as np

import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s %(message)s",
    datefmt="[%Y-%m-%d %H:%M:%S]",
)
logger = logging.getLogger(__name__)

def path_model(row):
    loc = row.loc["SaveLocation"]
    model_id = row.loc["model_id"]
    use_loss = row.loc["use_loss"]
    target_len = row.loc["target_len"]
    loc = Path(loc.replace("Trained", "UnzipTrained"))
    return loc/f"synthetic_out_{model_id}"/f"{use_loss}_{target_len}"

def model_artifacts(absolute_model_dir):
    """
    lists all files (with relative path to `dirname`) in `dirname`.
    """
    for root, _, files in os.walk(absolute_model_dir):
        for file in files:
            yield os.path.join(root, file)


def upload_dir_to_s3(absolute_model_dir, session, bucket, prefix):
    for file in model_artifacts(absolute_model_dir):
        suffix = "/".join(file.split("/")[7:])
        logger.info(f"{prefix}/{suffix}")
        session.upload_file(file, bucket, f"{prefix}/{suffix}")

def upload_model_s3(csv_file):
    df = pd.read_csv(csv_file)
    s3 = boto3.client('s3')
    Conditions_1 = namedtuple("Conditions_1", "TrainingJobStatus DownloadStatus SaveLocation")
    Conditions_2 = namedtuple("Conditions_2", "FileExists UploadS3")
    for idx, row in df.iterrows():
        logger.info(f"Iteration {idx+1}/{len(df)}")
        if idx%50==0:
            logger.info("Sleeping five seconds")
            time.sleep(5)
        TrainingJobName = row["TrainingJobName"]
        logger.info(f"Looking to upload {TrainingJobName}")
        cond_1 = Conditions_1(row["TrainingJobStatus"]=="Completed", row["DownloadStatus"], bool(row["SaveLocation"]))
        if all(cond_1):
            logger.info("First conditions are respected, let's check that the file exists locally and it has not already been uploaded")
            path = path_model(row)
            cond_2 = Conditions_2(path.exists(), not row["UploadS3"])
            if all(cond_2):
                logger.info("File exists and has not already been uploaded, uploading now")
                key = "ICLRModels"
                bucket_name = "s3-violation-test-bucket"
                upload_dir_to_s3(path, s3, bucket_name, f"{key}/{row.loc['TrainingJobName']}")
                logger.info(f"Successfully uploaded {TrainingJobName} under {bucket_name}/{key}")
                df.loc[idx, "UploadS3"] = True
                df.loc[idx, "UploadS3Location"] = f"{bucket_name}/{key}/{row.loc['TrainingJobName']}"
            else:
                logger.info(f"Could not upload {TrainingJobName}, conditions are not respected {cond_2}")
        else:
            logger.info(f"Could not upload {TrainingJobName}, all the conditions are not respected: {cond_1}")
            
            
#     print(df)
    df.to_csv(csv_file, index = False)
#                 print(path)

    
upload_model_s3("TrainingFollowUp.csv")
    


[2021-09-15 20:30:09] [INFO] __main__ Iteration 1/722
[2021-09-15 20:30:09] [INFO] __main__ Sleeping five seconds
[2021-09-15 20:30:14] [INFO] __main__ Looking to upload PSA-GAN-traffic-len16-29082021-010119-969
[2021-09-15 20:30:14] [INFO] __main__ Could not upload PSA-GAN-traffic-len16-29082021-010119-969, all the conditions are not respected: Conditions_1(TrainingJobStatus=False, DownloadStatus=False, SaveLocation=True)
[2021-09-15 20:30:14] [INFO] __main__ Iteration 2/722
[2021-09-15 20:30:14] [INFO] __main__ Looking to upload PSA-GAN-traffic-len32-29082021-010134-981
[2021-09-15 20:30:14] [INFO] __main__ Could not upload PSA-GAN-traffic-len32-29082021-010134-981, all the conditions are not respected: Conditions_1(TrainingJobStatus=False, DownloadStatus=False, SaveLocation=True)
[2021-09-15 20:30:14] [INFO] __main__ Iteration 3/722
[2021-09-15 20:30:14] [INFO] __main__ Looking to upload PSA-GAN-traffic-len64-29082021-010151-403
[2021-09-15 20:30:14] [INFO] __main__ Could not upload